# Neural Network From Scratch

In [1]:
import pandas as pd
import numpy as np
import time
import random
from MyDenceNN import MyDenceNN

random.seed(5)

# Initialise Network Parameters

In [2]:
def initialise_params(dims):
    """
    ## Generates units in each layer of the network ##
    Inputs: number of units in each layer dims = [n0, n1, n2, ..., nL]
    output: W = {"W1": W1, ..., "WL": WL}, b = {"b1": b1, ..., "bL": bL}
    """
    # Number of layers
    L = len(dims) - 1
    
    # Dictionary of parameters
    W = {}
    b = {}
    for l in range(1, L+1):
        W["W"+str(l)] = np.random.randn(dims[l], dims[l-1])
        b["b"+str(l)] = np.zeros((dims[l], 1))
    
    return W, b

In [3]:
# Test initialise_params()
dims = [2, 4, 4, 2]
W, b = initialise_params(dims)

print("------------------------------")
print("## Test initialise_params() ##")
print("------------------------------\n\n")

for l in range(1,len(dims)):
    print("Layer {}:".format(l))
    print("----------\n")
    print("W"+str(l)+" = {}".format(W["W"+str(l)]))
    print("b"+str(l)+" = {}\n".format(b["b"+str(l)]))

------------------------------
## Test initialise_params() ##
------------------------------


Layer 1:
----------

W1 = [[-0.52759779 -0.04665227]
 [-0.63599095 -0.27391402]
 [ 1.58062004 -0.52113235]
 [-1.20493073 -2.10624329]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]

Layer 2:
----------

W2 = [[ 1.41669001  0.23300962  1.10227609  0.0022881 ]
 [-0.10230629  0.42934874  0.06892114 -1.84443759]
 [ 0.21352441 -0.03239448 -0.87462826 -0.27457902]
 [ 0.50235315  0.36817753 -0.70550523  1.8896441 ]]
b2 = [[0.]
 [0.]
 [0.]
 [0.]]

Layer 3:
----------

W3 = [[0.26833982 1.25051952 0.60460984 0.52718498]
 [1.0078432  1.89942242 1.56969414 0.36078024]]
b3 = [[0.]
 [0.]]



# Forward Linear Calculation

In [4]:
def forward_linear(A_prev, W, b):
    """
    ## Calculate linear equation for forward pass ##
    Inputs: A = previous layer activation, W = current weights, b = current biases
    Output: Z = W•A + b
    """
    
    # Linear Equation
    Z = np.dot(W, A_prev) + b
    
    return Z

In [5]:
# Test forward_linear()
x = np.array([[2], [5]])
W1 = W["W1"]
b1 = b["b1"]

Z1 = forward_linear(x, W1, b1)

print("------------------------------")
print("## Test forward_linear() ##")
print("------------------------------\n")

print("\nInputs: \n----------")
print("x = ", x)
print("\nW1 = ", W1)
print("\nb1 = ", b1)

print("\nOutput: \n----------")
print("Z1 = ", Z1)

------------------------------
## Test forward_linear() ##
------------------------------


Inputs: 
----------
x =  [[2]
 [5]]

W1 =  [[-0.52759779 -0.04665227]
 [-0.63599095 -0.27391402]
 [ 1.58062004 -0.52113235]
 [-1.20493073 -2.10624329]]

b1 =  [[0.]
 [0.]
 [0.]
 [0.]]

Output: 
----------
Z1 =  [[ -1.28845696]
 [ -2.641552  ]
 [  0.55557832]
 [-12.94107788]]


# Forward Activation Calculation

In [6]:
########################
# Activation Functions #
########################
    

# Sigmodal Activation
def Sigmoid(Z):
    return 1/(1 + np.exp(-Z))

# Swish Activation
def Swish(Z):
    return Z/(1 + np.exp(-Z))

# Hyperbolic Tangent
def Tanh(Z):
    return np.tanh(Z)

# Rectified Linear Unit
def ReLU(Z):
    return np.maximum(0, Z)

# Leaky Rectified Linear Unit
def L_ReLU(Z):
    return np.maximum(0.01*Z, Z)

# # Parametric Rectified Linear Unit
# def P_ReLU(Z):
#     return np.maximum(0.05*Z, Z)


############################
# Forward pass calculation #
############################

def forward_activation(Z, g):
    """ 
    ## Applies specified activation function to Z ##
    Inputs: Z = linear forward pass, g = activation function
    Output: A = g(z) 
    """

    # Dictionary of activations functions
    activations = {
        'sigmoid': Sigmoid,
        'swish'  : Swish,
        'tanh'   : Tanh,
        'relu'   : ReLU,
        'l_relu' : L_ReLU
    }

    # Call activation on Z
    try:
        A = activations[g](Z)
        return A
    except:
        display(print("\n----------------\nInvalid activation function\n----------------\n"))
        display(print(g))
        display(print("----------------\nValid activations:\n----------------"))
        for key in activations.keys():
            display(print(key))
        pass

In [7]:
# Test forward_activation()
Z1 = Z1
g = "relu"

print("------------------------------")
print("## Test forward_activation() ##")
print("------------------------------\n")

print("\nInputs: \n----------")
print("Z1 = ", Z1)
print("\ng = ", g)

print("\nOutput: \n----------")
A1 = forward_activation(Z1, g)
print("A = ", A1)

------------------------------
## Test forward_activation() ##
------------------------------


Inputs: 
----------
Z1 =  [[ -1.28845696]
 [ -2.641552  ]
 [  0.55557832]
 [-12.94107788]]

g =  relu

Output: 
----------
A =  [[0.        ]
 [0.        ]
 [0.55557832]
 [0.        ]]


# Model Forward Propagation

In [8]:
def foward_propagation(X, W, b, g):
    """
    ## Completes a full forward pass through Network ##
    Inputs: X = Full Feature Set, W = Weights in Network, 
            b = Biases in Network, g = All Activation Functions
    Output: A, Z
    """
    assert(len(W) == len(b))
    assert(len(b) == len(g))
    assert(len(g) == len(W))
    
    L = len(W)
    A_prev = X
    A = {"A0": X}
    Z = {}
    
    for l in range(1, L+1):
        # Parameters
        bl = b["b"+str(l)]
        Wl = W["W"+str(l)]
        
        # Current activation 
        gl = g["g"+str(l)]
        
        # Forward Linear Pass
        Zl = forward_linear(A_prev, Wl, bl)
        Al = forward_activation(Zl, gl)
        
        # Chase Values
        A["A"+str(l)] = Al
        Z["Z"+str(l)] = Zl
        
        A_prev = Al   
    
    return A, Z

In [9]:
# Test forward_propagation()

## Specify inputs
X = x
dims = [X.shape[0], 4, 4, 2, 1]
g = {
    "g1": 'relu',
    "g2": 'relu',
    "g3": 'relu',
    "g4": 'sigmoid'
}

## Run initialise_params()
W, b = initialise_params(dims)

print("------------------------------")
print("## Test forward_activation() ##")
print("------------------------------\n\n")

print("N = ", dims)
print("-------------------\n-------------------\n\n")

## Run foward_propagation()
A, Z = foward_propagation(X, W, b, g)

for i in range(1,len(dims)):
    print("Layer {}:".format(i))
    print("----------\n")
    print("W"+str(i)+" = {}".format(W["W"+str(i)]))
    print("b"+str(i)+" = {}\n".format(b["b"+str(i)]))
    print("Z"+str(i)+" = {}".format(Z["Z"+str(i)]))
    print("A"+str(i)+" = {}\n\n".format(A["A"+str(i)]))

------------------------------
## Test forward_activation() ##
------------------------------


N =  [2, 4, 4, 2, 1]
-------------------
-------------------


Layer 1:
----------

W1 = [[-1.67472589  1.17168787]
 [-0.73413276  0.19163704]
 [ 1.56708526  1.28891966]
 [ 0.41414088  0.08694783]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]

Z1 = [[ 2.5089876 ]
 [-0.51008033]
 [ 9.57876882]
 [ 1.26302091]]
A1 = [[2.5089876 ]
 [0.        ]
 [9.57876882]
 [1.26302091]]


Layer 2:
----------

W2 = [[ 0.97038767 -0.39873249  0.07037556 -1.15446339]
 [ 0.30659523  0.64726699  0.08781716 -1.31573493]
 [ 0.33478087  0.61034402 -1.16348796  0.20223408]
 [-0.3366279   1.42573718  0.37038499  0.18312744]]
b2 = [[0.]
 [0.]
 [0.]
 [0.]]

Z2 = [[  1.65069046]
 [ -0.05137686]
 [-10.04939529]
 [  2.93453074]]
A2 = [[1.65069046]
 [0.        ]
 [0.        ]
 [2.93453074]]


Layer 3:
----------

W3 = [[-1.58188333  0.26071975 -0.28425539 -0.27306191]
 [-1.37581046 -0.69379836 -1.957115    2.62604198]]
b3 = [[0.]
 [0.]]

Z

# Calculate Cost

In [10]:
def cost(AL, Y, cost_fun):
    """
    Calculates the current cost (error) of the models out puts AL
    Inputs: AL = Model Predication, Y = Expected (True) values, cost_fun = Specified Cost Function
    Output: J = cost
    """
    
    # List of available functions
    functions = {## Add more functionlater
        'logistic_regression': LOG, # Logistic Regression
        # 'mean_squared_error' : MSE, # Mean Squared Error
        # 'mean_absolute_error': MAE  # Mean Absolute Error
    }
    
    try:
        J = functions[cost_fun](AL, Y)
        return J
    except:
        display(print("\n----------------\nInvalid Cost Function\n----------------\n"))
        display(print(cost_fun))
        display(print("----------------\nValid Cost Functions:\n----------------"))
        for key in activations.keys():
            display(print(key))
        pass
        


##################
# Cost functions #
##################

# Logistic Regression
def LOG(AL, Y):
    J = -1/Y.shape[1]*np.sum(Y*np.log(AL) + (1-Y)*np.log(1-AL), axis=1, keepdims=True)
    return J

# Mean Squared Error
def MSE(AL, Y):
    error = Y - AL
    J = -1/Y.shape[1]*np.sum(error*error, axis=1, keepdims=True)
    return J

# Mean Absolute Error
def MAE(AL, Y):
    J = -1/Y.shape[1]*np.sum(np.absolute(Y, AL), axis=1, keepdims=True)
    return J

In [11]:
# Test cost()

## Specify inputs
X = x
Y = np.array([[1]])
dims = [X.shape[0], 4, 4, 2, Y.shape[1]]
L = len(dims) - 1
g = {
    "g1": 'relu',
    "g2": 'relu',
    "g3": 'relu',
    "g4": 'sigmoid'
}
cost_fun = 'logistic_regression'

## Run initialise_params()
W, b = initialise_params(dims)

print("-------------------")
print("## Test cost() ##")
print("-------------------\n\n")

print("\nInputs Test Data: \tX = ", X)
print("\nExpected Outputs: \tY = ", Y)
print("\nUnits in each layer: \tN = ", dims)
print("\nNumber of Layers: \tL = ", L)
print("-------------------\n-------------------\n\n")

## Run foward_propagation()
A, Z = foward_propagation(X, W, b, g)

## Run cost()
J = cost(A["A"+str(L)], Y, cost_fun)

for i in range(1, L+1):
    print("Layer {}:".format(i))
    print("----------\n")
    print("W"+str(i)+" = {}".format(W["W"+str(i)]))
    print("b"+str(i)+" = {}\n".format(b["b"+str(i)]))
    print("Z"+str(i)+" = {}".format(Z["Z"+str(i)]))
    print("A"+str(i)+" = {}\n\n".format(A["A"+str(i)]))

print("--------------------------------------\n--------------------------------------")
print("##### Cost: J = {} ####".format(J))
print("--------------------------------------\n--------------------------------------")

-------------------
## Test cost() ##
-------------------



Inputs Test Data: 	X =  [[2]
 [5]]

Expected Outputs: 	Y =  [[1]]

Units in each layer: 	N =  [2, 4, 4, 2, 1]

Number of Layers: 	L =  4
-------------------
-------------------


Layer 1:
----------

W1 = [[-0.21901098 -0.45797075]
 [-0.87272402 -1.869352  ]
 [-1.8785267  -0.42425082]
 [-0.37559362  0.07861534]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]

Z1 = [[ -2.72787571]
 [-11.09220806]
 [ -5.87830752]
 [ -0.35811053]]
A1 = [[0.]
 [0.]
 [0.]
 [0.]]


Layer 2:
----------

W2 = [[ 0.76364955  0.18902406 -1.29198788 -0.0217373 ]
 [ 1.65281356 -1.25447408 -0.53294204  0.7844104 ]
 [ 1.26578813  0.35131684  0.10182487  0.91627151]
 [ 1.4131937  -1.04739154 -0.12172404  0.98545104]]
b2 = [[0.]
 [0.]
 [0.]
 [0.]]

Z2 = [[0.]
 [0.]
 [0.]
 [0.]]
A2 = [[0.]
 [0.]
 [0.]
 [0.]]


Layer 3:
----------

W3 = [[ 0.95678973  0.27632893  0.22575616  0.22424986]
 [-0.8731264   0.33624783 -1.02009524  0.38222052]]
b3 = [[0.]
 [0.]]

Z3 = [[0.]
 [0.]]
A3

# Initialise Backpropagation

In [12]:
# def initalise_dAL(AL, g, cost_fun):
#     """
#     Calculate dAL = dJ/dAL
#     """
    
#     # List of available functions
#     dFunctions = {## Add more functionlater
#         'logistic_regression': dLOG, # Logistic Regression
#         # 'mean_squared_error' : dMSE, # Mean Squared Error
#         # 'mean_absolute_error': dMAE  # Mean Absolute Error
#     }
    
#     J = dFunctions[cost_fun](AL, Y)
    
#     return dAL


In [13]:
# Test initial_dAL()


# Backward Activation Calculation

In [14]:
def Sigmoid_prime(A):
    return A * (1-A)

def Tanh_prime(A):
	return 1 - np.power(A, 2)

def ReLU_prime(A):
    return 1 if A > 0 else 0

def L_ReLU_prime(A, alpha):
	return 1 if A > 0 else alpha

def initial_dAL(AL, Y):
    dAL = - (Y/AL - (1 - Y)/(1 - AL))
    
    return dAL



def backward_activation(Wl, bl, Al, dAl, A_prev, gl):
    """
    
    """
    
    # Dictionary of activations functions
    derivatives = {
        'sigmoid': Sigmoid_prime,
        'tanh'   : Tanh,
        'relu'   : ReLU_prime,
        'l_relu' : L_ReLU_prime
    }
    
    m = Al.shape[1]
    gl_prime = derivatives[gl](Al)
        
    dZl = dAl*gl_prime
        
    dWl = 1/m*np.dot(dZl, A_prev.T)
    
    dbl = 1/m*np.sum(dZl, axis=1, keepdims=True)
        
    # Calculate dA[l-1]
    dA_prev = np.dot(Wl.T, dZl)
    
    return dA_prev, dWl, dbl

In [15]:
# Test initial_dAL()
A_prev = A["A"+str(L-1)]
AL = A["A"+str(L)]
gL = g["g"+str(L)]
WL = W["W"+str(L)]
bL = b["b"+str(L)]
Y = Y
dAL = initial_dAL(AL, Y)

print(A_prev)
print(AL)
print(WL)
print(bL)
print(gL)
print(Y)
print(dAL)

dA_prev, dWL, dbL = backward_activation(WL, bL, AL, dAL, A_prev, gL)

print(dA_prev)
print(dWL)
print(dbL)

[[0.]
 [0.]]
[[0.5]]
[[ 1.20823389 -1.91686432]]
[[0.]]
sigmoid
[[1]]
[[-2.]]
[[-0.60411695]
 [ 0.95843216]]
[[0. 0.]]
[[-0.5]]


# Backward Linear Calculation

In [16]:
def backward_linear(dW, db, W, b, alpha=1):
    
    W -= alpha*dW
    b -= alpha*db
    return W, b

In [17]:
# Test backwards_linear()
alpha = 0.1
WL_updated, bL_updated= backward_linear(dWL, dbL, WL, bL, alpha)

print(WL_updated)
print(bL_updated)

[[ 1.20823389 -1.91686432]]
[[0.05]]


# Model Backward Propagation

In [18]:
def back_propagation(W, b, AL, Y, cost_fun, g):
    """
    
    """
    
    # Initialoise
    L = len(W)
    A_prev = A["A"+str(L)]
    dA_prev = initial_dAL(AL, Y)
    
    # Backpropegate
    for l in range(L, 0, -1):
        # Get Variable and Parameters
        Wl, bl, gl = W["W"+str(l)], b["b"+str(l)], g["g"+str(l)]
        Al, dAl = A_prev, dA_prev
        A_prev  = A["A"+str(l-1)]
        
        # Calculate derivates
        dA_prv, dWl, dbl = backward_activation(WL, bL, AL, dAL, A_prev, gl)
        
        # Update current Parameters
        W["W"+str(l)], b["b"+str(l)] = backward_linear(dWl, dbl, Wl, bl, .01)
        
        
    
    return W, b
    

In [19]:
# Test back_propagation()

## Specify inputs
X = np.array([[2], [5]])
Y = np.array([[1]])
dims = [X.shape[0], 4, 4, 2, Y.shape[1]]
L = len(dims) - 1
g = {
    "g1": 'relu',
    "g2": 'relu',
    "g3": 'relu',
    "g4": 'sigmoid'
}
cost_fun = 'logistic_regression'

## Run initialise_params()
W, b = initialise_params(dims)

print("-------------------")
print("## Test cost() ##")
print("-------------------\n\n")

print("\nInputs Test Data: \tX = ", X)
print("\nExpected Outputs: \tY = ", Y)
print("\nUnits in each layer: \tN = ", dims)
print("\nNumber of Layers: \tL = ", L)
print("-------------------\n-------------------\n\n")

## Run foward_propagation()
A, Z = foward_propagation(X, W, b, g)

## Run cost()
J = cost(A["A"+str(L)], Y, cost_fun)

for i in range(1, L+1):
    print("Layer {}:".format(i))
    print("----------\n")
    print("W"+str(i)+" = {}".format(W["W"+str(i)]))
    print("b"+str(i)+" = {}\n".format(b["b"+str(i)]))
    print("Z"+str(i)+" = {}".format(Z["Z"+str(i)]))
    print("A"+str(i)+" = {}\n\n".format(A["A"+str(i)]))

print("--------------------------------------\n--------------------------------------")
print("##### Cost: J = {} ####".format(J))
print("--------------------------------------\n--------------------------------------")

W, b = back_propagation(W, b, AL, Y, cost_fun, g)
# Print updates
PRINT = True

for l in range(L, 0, -1):
    print("\nLayer {}:".format(l))
    print("----------\n")
    print("W"+str(l)+" = {}".format(W["W"+str(l)]))
    print("b"+str(l)+" = {}\n".format(b["b"+str(l)]))

-------------------
## Test cost() ##
-------------------



Inputs Test Data: 	X =  [[2]
 [5]]

Expected Outputs: 	Y =  [[1]]

Units in each layer: 	N =  [2, 4, 4, 2, 1]

Number of Layers: 	L =  4
-------------------
-------------------


Layer 1:
----------

W1 = [[ 1.49116479 -2.21504256]
 [-0.17000539 -0.80244003]
 [ 1.60671005  0.10759652]
 [-0.43025835  0.35524553]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]

Z1 = [[-8.09288323]
 [-4.35221095]
 [ 3.75140269]
 [ 0.91571094]]
A1 = [[0.        ]
 [0.        ]
 [3.75140269]
 [0.91571094]]


Layer 2:
----------

W2 = [[ 0.62122217  0.81608305  1.44128942 -0.63473784]
 [ 0.30913332 -0.00746083  1.73183946 -0.77769518]
 [-1.02737469  0.49502795 -0.13817145 -0.66329786]
 [-0.30714873 -1.22141762  0.26524205  0.36767423]]
b2 = [[0.]
 [0.]
 [0.]
 [0.]]

Z2 = [[ 4.82562062]
 [ 5.78468322]
 [-1.12572585]
 [ 1.33171307]]
A2 = [[4.82562062]
 [5.78468322]
 [0.        ]
 [1.33171307]]


Layer 3:
----------

W3 = [[-0.11741849  0.06480004 -0.53672602  0.48352